In [1]:
import sklearn
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.linear_model import Lasso

In [2]:
df = pd.read_csv('train.csv')
df = df.drop(['id'],axis=1)
df = pd.get_dummies(df, columns=['penalty'])
X = df.drop(['time'],axis=1)
y = df.loc[:,'time']


In [3]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler


In [4]:
kf = KFold(n_splits=4, shuffle=True)
scaler = StandardScaler()

X_ = X.loc[:, ['penalty_l2', 'penalty_none', 'penalty_elasticnet',  'max_iter','n_jobs', 'n_samples', 'n_features', 'n_classes', 'flip_y']]
X_['job_inv'] = 1 / X_['n_jobs']
X_['workload'] = X_['max_iter'] * X_['n_classes']  * X_['n_samples'] * X_['n_features'] * X_['flip_y'] * X_['job_inv']
X_['class_job'] = X_['n_classes'] * X_['job_inv']
X_['samples_job'] = X_['n_samples'] * X_['job_inv']
X_['iter_job'] = X_['max_iter'] * X_['job_inv']
X_['feature_job'] = X_['n_features'] * X_['job_inv']
X_ = scaler.fit_transform(X_)

for train_index, test_index in kf.split(df):
    y_train, y_test = y.iloc[train_index,], y.iloc[test_index,]
    X_train, X_test = X_[train_index], X_[test_index]
    log_y_train = np.log(y_train)
    est = GradientBoostingRegressor(n_estimators=1000, max_features=0.5, subsample=0.5, learning_rate=0.01, max_depth=3, random_state=0, loss='ls')
    est.fit(X_train, log_y_train, sample_weight=y_train )
    
    print(mean_squared_error(y_test, np.exp(est.predict(X_test))))
    print(mean_squared_error(y_train, np.exp(est.predict(X_train))))

3.303476163636026
0.06971091730184685
2.441950815242664
0.09175120100820375
3.6430731221882233
0.0669859100168007
5.160245000373136
0.07697454248099596
